In [ ]:
import pandas as pd
import json
from pathlib import Path

import networkx as nx
import matplotlib.pyplot as plt

In [ ]:
AFFILIATIONS = ["UGM", "UI", "ITB", "IPB", "UB"]
for affiliation in AFFILIATIONS:
    with open(f"../data/processed/{affiliation}/life_science.json", "r") as f:
        data.update(json.load(f))

In [ ]:


# Initialize a new directed graph
G = nx.DiGraph()

for sinta_id, value in data.items():
    # Add the main author to the graph
    main_author_id = data[sinta_id]["scholar_id"]
    if main_author_id != None:
        G.add_node(main_author_id, role='main', name=value['name_inputted'], affiliation=value['affiliation_sinta']['name'])
    else:
        continue

    # Add the co-authors to the graph
    for coauthor in data[sinta_id]['coauthors']:
        coauthor_id = coauthor['scholar_id']
        G.add_node(coauthor_id, role='coauthor', name=coauthor["name"], affiliation=coauthor["affiliation"])

        # Add an edge from the main author to the co-author
        G.add_edge(main_author_id, coauthor_id)

for sinta_id, value in data.items():
    # Add the main author to the graph
    main_author_id = data[sinta_id]["scholar_id"]
    if main_author_id != None:
        G.nodes[main_author_id]["role"] = "main"
        G.nodes[main_author_id]["name"] = value['name_inputted']
        G.nodes[main_author_id]["affiliation_sinta"] = value['affiliation_sinta']['name']
    else:
        continue

# Create a dictionary of labels
labels = {node: G.nodes[node]["name"] for node, data in G.nodes(data=True) if data['role'] == 'main'}

# Create a color map
color_map = ['red' if data['role'] == 'main' else 'gray' for node, data in G.nodes(data=True)]

# Create a size map
size_map = [100 if data['role'] == 'main' else 25 for node, data in G.nodes(data=True)]

# Draw the graph
pos = nx.kamada_kawai_layout(G)
nx.draw(G, pos, node_color=color_map, node_size=size_map, alpha=0.5, with_labels=False)  # Draw nodes and edges

# Draw labels
nx.draw_networkx_labels(G, pos, labels, font_size=5)

# Save the plot as a file
plt.savefig(f'../figures_ALL/02.0_co-author_graph.png')

# Show the plot
plt.show()

In [ ]:
outfile = Path(f"../data/processed/ALL/co-author_network.graphml")
outfile.parent.mkdir(exist_ok=True, parents=True)
nx.write_graphml(G, outfile)

In [ ]:
main_authors = []
connecting_co_authors = []
for node in G.nodes:
    role = G.nodes[node]["role"]
    if role == "main":
        main_authors.append(node)
    else:
        first_neighbors = [n for n in nx.all_neighbors(G, node)]
        if len(first_neighbors) > 1:
            connecting_co_authors.append(node)

In [ ]:
# Create a list of nodes for the subgraph
nodes = list(main_authors) + list(connecting_co_authors)

# Create the subgraph
H = G.subgraph(nodes)

# Create a dictionary of labels
labels = {node: H.nodes[node]["name"] for node, data in H.nodes(data=True) if data['role'] == 'main'}

# Create a color map
color_map = ['red' if data['role'] == 'main' else 'gray' for node, data in H.nodes(data=True)]

# Create a size map
size_map = [100 if data['role'] == 'main' else 25 for node, data in H.nodes(data=True)]

# Draw the graph
pos = nx.kamada_kawai_layout(H)
nx.draw(H, pos, node_color=color_map, node_size=size_map, alpha=0.5, with_labels=False)  # Draw nodes and edges

# Draw labels
nx.draw_networkx_labels(H, pos, labels, font_size=5)

# Save the plot as a file
plt.savefig('../figures_ALL/02.2_connecting_co-author_graph.png')

# Show the plot
plt.show()

In [ ]:
nx.write_graphml(H, "../data/processed/ALL/co-author_subgraph_network.graphml")

In [ ]:
value['affiliation_sinta']['name']

In [ ]:
value